General DESI related packages

In [1]:
# general packages
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import specsim
import specsim.quickspecsim as qspecsim
import specsim.instrument as inst
import specsim.source as src
import specsim.config as conf
import specsim.fiberloss as floss
import specsim.observation as obs
import specsim.simulator as sim
import specsim.atmosphere as atm

from desisim import io
from desisim import obs

import astropy.units as u
import astropy.table

In [7]:
num_source = 1
DEBUG      = True
desi_configs = ['desi-blur', 'desi-blur-offset', 'desi-blur-offset-scale', 'desi-blur-offset-scale-stochastic']
config_path = '/home/tyapici/Projects/DESI_projects/dithering/config/'
desi_models = {}

# generate all relevant configurations for future use
for config in desi_configs:
    desi_models[config] = desi_model(config)

# default model is set to desi+blur+offset
curr_desi_model_string = 'desi-blur-offset'
curr_desi_model = desi_models[curr_desi_model_string]

# the source information will be used later on for the fiber loss calculation
# using their expected spectrum
# Round sources are generated here
qsos_round = []
elgs_round = []
lrgs_round = []
for i in range(num_source):
    qsos_round.append(generate_sources(1, 0., 0., seed=23+i))
    elgs_round.append(generate_sources(1, 1., 0., seed=23+i))
    lrgs_round.append(generate_sources(1, 0., 1., seed=23+i))
# elliptic objects are generated here
#qsos_elliptic = generate_sources(num_source, 0., 0., seed=1, minormajor=[0.4, 0.6])
#elgs_elliptic = generate_sources(num_source, 1., 0., seed=1, minormajor=[0.4, 0.6])
#lrgs_elliptic = generate_sources(num_source, 0., 1., seed=1, minormajor=[0.4, 0.6])
print("round and elliptic sources has been generated")

round and elliptic sources has been generated


In [2]:
# run the specsim.simulator with a configuration at "model"
def desi_model(model):
    return sim.Simulator(config_path+model+'.yaml')

In [8]:
# function to generate the wavelength values
def create_wlen_grid(num_wlen=11, desi=curr_desi_model):
    wavelength = desi.simulated['wavelength']
    wlen_unit = wavelength.unit
    return np.linspace(wavelength.data[0], wavelength.data[-1], num_wlen) * wlen_unit

In [9]:
# function to generate the positions of the fibers.
# TODO: it is done with random positions, need to change with the actual positions
def generate_fiber_positions(nfiber=5000, seed=13, desi=curr_desi_model, isRandom=True):
    if isRandom:
        gen = np.random.RandomState(seed)
        focal_r = (np.sqrt(gen.uniform(size=nfiber)) * desi.instrument.field_radius)
        phi = 2 * np.pi * gen.uniform(size=nfiber)
    else:
        # this part is broken. needs some fixing
        gen = np.random.RandomState(seed)
        focal_r = (np.sqrt(gen.uniform(0, 0.001, size=nfiber)) * desi.instrument.field_radius)
        phi = 2 * np.pi * gen.uniform(size=nfiber)
    return np.cos(phi) * focal_r, np.sin(phi) * focal_r

In [10]:
# function to plot the fiber positions nicely
def plot_fiber_positions(focal_x, focal_y, save=None, desi=curr_desi_model, select=None):
    plt.figure(figsize=(8, 8))
    x, y = focal_x.to(u.mm).value, focal_y.to(u.mm).value
    plt.plot(x, y, '.')
    if select is not None:
        plt.scatter(x[select], y[select], s=100)
    r = 1.05 * desi.instrument.field_radius.to(u.mm).value
    plt.xlim(-1.01 * r, +1.01 * r)
    plt.ylim(-1.01 * r, +1.01 * r)
    plt.gca().set_aspect('equal', 'datalim')
    plt.gca().add_artist(plt.Circle((0, 0), r, color='r', fill=False))
    plt.axis('off')
    if save:
        plt.savefig(save)
    else:
        plt.show()

In [6]:
# function to generate mock sources
def generate_sources(nsrc=5000, disk_fraction=1., bulge_fraction=0., vary='', seed=23, minormajor=[1,1]):
    gen = np.random.RandomState(seed)
    varied = vary.split(',')
    source_fraction = np.tile([disk_fraction, bulge_fraction], (nsrc, 1))
    source_half_light_radius = np.tile([0.45, 1.0], (nsrc, 1))
    source_minor_major_axis_ratio = np.tile(minormajor, (nsrc, 1))
    #if 'pa' in varied:
    source_position_angle = 360. * gen.uniform(size=(nsrc, 2))
    #else:
    #    source_position_angle = np.tile([0., 0.], (nsrc, 1))
    return source_fraction, source_half_light_radius, source_minor_major_axis_ratio, source_position_angle

In [11]:
def calculateFiberLoss(wlen_grid, desi=curr_desi_model):
    calc = floss.GalsimFiberlossCalculator(desi.instrument.fiber_diameter.to(u.um).value,
                                           wlen_grid.to(u.Angstrom).value,
                                           num_pixels=16, oversampling=32, moffat_beta=3.5)
    return calc

In [12]:
def get_fiberloss(source_fraction, source_half_light_radius, source_minor_major_axis_ratio,
                  source_position_angle,
                  x, y, wlen_grid, seeing=1.1*u.arcsec, desi=curr_desi_model):
    # Subtract instrumental PSF.
    #Jacoby = 0.219
    #seeing = 2.35482 * np.sqrt((seeing.to(u.arcsec).value/2.35482) ** 2 - Jacoby**2) * u.arcsec
    # Tabulate seeing.
    #desi.atmosphere.seeing_fwhm_ref = seeing
    seeing_fwhm = desi.atmosphere.get_seeing_fwhm(wlen_grid).to(u.arcsec).value
    # Calculate optics.
    # NOTE: TODO: may add dithering stuff here
    
    scale, blur, offset = desi.instrument.get_focal_plane_optics(x, y, wlen_grid)
    print(offset)
    print(offset.shape)
    print(wlen_grid)
    # Do the fiberloss calculations.
    calc = calculateFiberLoss(wlen_grid, desi=desi)
    print("i am here")
    print(calc.calculate(seeing_fwhm,
                          scale.to(u.um / u.arcsec).value, offset.to(u.um).value,
                          blur.to(u.um).value,
                          source_fraction, source_half_light_radius,
                          source_minor_major_axis_ratio, source_position_angle))
    return calc.calculate(seeing_fwhm,
                          scale.to(u.um / u.arcsec).value, offset.to(u.um).value,
                          blur.to(u.um).value,
                          source_fraction, source_half_light_radius,
                          source_minor_major_axis_ratio, source_position_angle)
    # there is something going wrong here with the source position angle
    # no matter what it is, the plots look the same for different sources at different locations

In [13]:
def get_desimodel_fiberloss(obj_type='lrg'):
    path = os.path.join(os.environ['DESIMODEL'], 'data', 'throughput',
                        'fiberloss-{0}.dat'.format(obj_type))
    t = astropy.table.Table.read(path, format='ascii', names=['wlen', 'accept'])
    return t

In [14]:
def plot_fiberloss(floss, cmap_name='viridis', alpha=0.1, ylim=(None, None),
                   desi=curr_desi_model, desimodel_type=None, save=None, overlay=False):
    if not overlay:
        plt.figure(figsize=(5, 4))
    wlen = wlen_grid.value
    rvalues = np.sqrt(focal_x ** 2 + focal_y ** 2)
    norm = matplotlib.colors.Normalize(
        vmin=0., vmax=desi.instrument.field_radius.to(u.mm).value)
    sm = matplotlib.cm.ScalarMappable(norm=norm, cmap=plt.get_cmap(cmap_name))
    for f, r in zip(floss, rvalues):
        color = sm.to_rgba(r)
        p = plt.plot(wlen, f, ls='-', c=color, alpha=0.2)
    plt.plot(wlen, np.median(floss, axis=0), 'k-', lw=2, label='median')
    if desimodel_type:
        t = get_desimodel_fiberloss(desimodel_type)
        plt.plot(t['wlen'], t['accept'], 'r--', lw=2, label='desimodel')
    if not overlay:
        plt.ylim(*ylim)
        # Very useful trick from http://stackoverflow.com/posts/11558629/revisions
        sm._A = []
        plt.colorbar(sm).set_label('Radius [mm]')
        plt.xlabel('Wavelength [Angstrom]')
        plt.ylabel('Fiber accepetance fraction')
        plt.xlim(wlen[0], wlen[-1])
        plt.tight_layout()
    if save:
        plt.savefig(save)

In [15]:
def plot_one_fiberloss(floss, ix, iy, wlen_grid, cmap_name='viridis', alpha=0.1, ylim=(None, None),
                       desi=curr_desi_model, desimodel_type=None, save=None, overlay=False):
    wlen = wlen_grid.value
    line_median, = plt.plot(wlen, np.median(floss, axis=0), 'k-', lw=2, label='dx=%d dy=%d'%(ix*5, iy*5), alpha=0.1)
    return line_median

In [16]:
def run(fiber_pos_seed, focal_x, focal_y, source_index=0):
    print(*qsos_round[source_index])
    # generate the wavelengths array
    wlen_grid = create_wlen_grid()
    print("wavelength grid has been generated")
    
    # plot the position of fibers
    # number of fibers needs to be equal to the number of sources
    # currently, there is only one source (that makes it easy to run a test)
    # TODO: make the whole script working for more than one fiber 
    if DEBUG:
        plot_fiber_positions(focal_x, focal_y, save="figures/fiber_pos_%d.png"%fiber_pos_seed)
        print("fiber position has been plotted")

    desi = curr_desi_model
    f_qso_round  = get_fiberloss(*qsos_round[source_index], focal_x, focal_y, wlen_grid, desi=desi)
    #f_elgs_round = get_fiberloss(*elgs_round[source_index], focal_x, focal_y, wlen_grid, desi=desi)
    #f_qso_elliptic  = get_fiberloss(*qsos_elliptic, focal_x, focal_y, wlen_grid, desi=desi)
    #f_elgs_elliptic = get_fiberloss(*elgs_elliptic, focal_x, focal_y, wlen_grid, desi=desi)
    #f_lrgs_elliptic = get_fiberloss(*lrgs_elliptic, focal_x, focal_y, wlen_grid, desi=desi)
    print("fiber loss calculations finished")
    
    x = []
    y = []
    w_3550 = []
    w_6700 = []
    w_9850 = []
    we_3550 = []
    we_6700 = []
    we_9850 = []
    num_dither = 80
    dither = np.linspace(-10, 10, num_dither)
    plt.clf()
    for ix in range(num_dither):
        for iy in range(num_dither):
            f_qso_round  = get_fiberloss(*qsos_round[source_index], focal_x+dither[ix]*5*u.mm, focal_y+dither[iy]*5.*u.mm, wlen_grid, desi=desi)
            x.append(dither[ix]*5)
            y.append(dither[iy]*5)
            w_3550.append(np.array(f_qso_round).flatten()[0])
            w_6700.append(np.array(f_qso_round).flatten()[5])
            w_9850.append(np.array(f_qso_round).flatten()[-1])
            if ix==0 and iy==0:
                wc_3550 = np.array(f_qso_round).flatten()[0]
                wc_6700 = np.array(f_qso_round).flatten()[5]
                wc_9850 = np.array(f_qso_round).flatten()[-1]
            l1 = plot_one_fiberloss(f_qso_round, ix, iy, wlen_grid, ylim=(0.18, 0.68), desimodel_type='qso')#, save="figures/qso_round_fiberloss_%d.png"%(dither[i]))
    # TODO: still this is hardcoded and only doing qso's. need to change it or add more here to
    #       accomomodate other sources
    t = get_desimodel_fiberloss('qso')
    xs = np.array(t['wlen'])
    ys = np.array(t['accept'])
    model_acceptance_func = interp1d(xs, ys, fill_value='extrapolate')
    we_3550 = model_acceptance_func(3550)
    we_6700 = model_acceptance_func(6700)
    we_9850 = model_acceptance_func(9850)
    line_model, = plt.plot(t['wlen'], t['accept'], 'r--', lw=2, label='desimodel')
    # TODO: depending on the source type the filenames here and after should be changed.
    plt.savefig("figures/ugly_plot_%s_%d_%d.png"%(curr_desi_model_string, fiber_pos_seed, source_index))
    print("arrays have been filled... Starting 2D histogram generation")
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], bins=num_dither, weights=np.array(w_3550))
    plt.colorbar()
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 3550 A\nExpected Acceptance %f\nCenter Value %f"%(we_3550, wc_3550))
    plt.savefig("figures/dither_fiber_acceptance_3550_%s_%d_%d.png"%(curr_desi_model_string, fiber_pos_seed, source_index))
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], bins=num_dither, weights=np.array(w_6700))
    plt.colorbar()
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 6700 A\nExpected Acceptance %f\nCenter Value %f"%(we_6700, wc_6700))
    plt.savefig("figures/dither_fiber_acceptance_6700_%s_%d_%d.png"%(curr_desi_model_string, fiber_pos_seed, source_index))
    
    plt.clf()
    plt.hist2d(np.array(x), np.array(y), range=[[-50, 50], [-50, 50]], bins=num_dither, weights=np.array(w_9850))
    plt.colorbar()
    plt.xlabel("X [mm]")
    plt.ylabel("Y [mm]")
    plt.suptitle("Fiber Acceptance at 9850 A\nExpected Acceptance %f\nCenter Value %f"%(we_9850, wc_9850))
    plt.savefig("figures/dither_fiber_acceptance_9850_%s_%d.png"%(curr_desi_model_string, fiber_pos_seed))
    
    print("Finished analysis for this configuration")
    print("You can continue with the next iteration")
    
    # TODO: this function should return some values for further evaluation
    #       besides plotting somethings

In [17]:
iterations   = 20
random_seeds = []
focal_pos    = []
for i in range(iterations):
    # fiber positions
    random_integer = np.random.randint(0, 1000)
    focal_pos.append(generate_fiber_positions(num_source, isRandom=True, seed=random_integer))
    random_seeds.append(random_integer)
np.array(random_seeds)
print("Random numbers have been generated for fiber position")
print("Fiber positions have been assigned")

Random numbers have been generated for fiber position
Fiber positions have been assigned


In [ ]:
for i in range(1):
    run(random_seeds[i], *focal_pos[i], source_index=0)
    run(random_seeds[i], *focal_pos[i], source_index=1)
    try:
        run(random_seeds[i], *focal_pos[i], source_index=0)
        run(random_seeds[i], *focal_pos[i], source_index=1)
    except:
        continue

[[ 0.  0.]] [[ 0.45  1.  ]] [[1 1]] [[ 186.22723818  340.90653737]]
wavelength grid has been generated
fiber position has been plotted
[[[  8.34446739  13.63615251]
  [  8.95151617  13.63615251]
  [  4.9435771   13.63615251]
  [  0.27827077  13.63615251]
  [ -3.22168269  13.63615251]
  [ -5.00907224  13.63615251]
  [ -5.26525276  13.63615251]
  [ -4.15707851  13.63615251]
  [ -1.8464154   13.63615251]
  [  0.58398883  13.63615251]
  [  3.01439307  13.63615251]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.52866717  0.54086744  0.56511065  0.58380152  0.59143537  0.59340616
   0.59512274  0.59877221  0.60462372  0.61081599  0.61498073]]
i am here
fiber loss calculations finished
[[[ 11.55426692  -6.5992391 ]
  [ 11.69927183  -6.5992391 ]
  [  6.92276371  -6.5992391 ]
  [  1.49131377  -6.5992391 ]
  [ -2.62817096  -6.5992391 ]
  [ -4.86956425  -6.5992391 ]
  [ -5.44554679  -6.5992391 ]
  [ -4.54462844  -6

[[[ 5.64303592 -3.89470045]
  [ 6.23984939 -3.89470045]
  [ 2.21488479 -3.89470045]
  [-2.46739334 -3.89470045]
  [-5.98107081 -3.89470045]
  [-7.77851757 -3.89470045]
  [-8.04178242 -3.89470045]
  [-6.93819935 -3.89470045]
  [-4.62982321 -3.89470045]
  [-2.19994404 -3.89470045]
  [ 0.22993512 -3.89470045]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.55006077  0.56342207  0.5864619   0.60303661  0.60876222  0.60970647
   0.61136832  0.61592523  0.62360349  0.63178497  0.63793036]]
i am here
[[[ 6.61779914 -1.8168692 ]
  [ 7.24912995 -1.8168692 ]
  [ 3.28158187 -1.8168692 ]
  [-1.34346092 -1.8168692 ]
  [-4.81085584 -1.8168692 ]
  [-6.5743859  -1.8168692 ]
  [-6.8137597  -1.8168692 ]
  [-5.69469343 -1.8168692 ]
  [-3.3786048  -1.8168692 ]
  [-0.94695488 -1.8168692 ]
  [ 1.48469503 -1.8168692 ]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am h

i am here
[[ 0.55173002  0.56932026  0.58661676  0.59713279  0.5997186   0.60025045
   0.60313095  0.60991621  0.6205072   0.63132642  0.6401317 ]]
i am here
[[[ -3.85370036  -2.89521902]
  [ -1.89264772  -2.89521902]
  [ -4.55466326  -2.89521902]
  [ -8.05512428  -2.89521902]
  [-10.67993191  -2.89521902]
  [-11.84019507  -2.89521902]
  [-11.65293471  -2.89521902]
  [-10.24808922  -2.89521902]
  [ -7.7754791   -2.89521902]
  [ -5.29012752  -2.89521902]
  [ -2.80477594  -2.89521902]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.55166808  0.5694038   0.58693965  0.59783331  0.60080818  0.60160815
   0.60459483  0.61131306  0.6216638   0.63219701  0.64069963]]
i am here
[[[ -3.73292458  -2.9678193 ]
  [ -1.68068309  -2.9678193 ]
  [ -4.25461384  -2.9678193 ]
  [ -7.67967661  -2.9678193 ]
  [-10.24820127  -2.9678193 ]
  [-11.36821545  -2.9678193 ]
  [-11.1524849   -2.9678193 ]
  [ -9.72853683  -2.9678193 ]

   0.60678535  0.61422709  0.62436918  0.63419966  0.64190334]]
i am here
[[[ -8.82619333   4.43757014]
  [ -5.42071257   4.43757014]
  [ -6.68746798   4.43757014]
  [ -8.99362312   4.43757014]
  [-10.7269105    4.43757014]
  [-11.249631     4.43757014]
  [-10.61140396   4.43757014]
  [ -8.90397414   4.43757014]
  [ -6.26489139   4.43757014]
  [ -3.72157224   4.43757014]
  [ -1.17825308   4.43757014]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.54206206  0.5659031   0.58363384  0.59489338  0.5994863   0.60232746
   0.60699118  0.61465458  0.62506583  0.63513521  0.64307782]]
i am here
[[[-10.03098874   3.74009981]
  [ -6.53633146   3.74009981]
  [ -7.71694588   3.74009981]
  [ -9.94936662   3.74009981]
  [-11.62761306   3.74009981]
  [-12.11097277   3.74009981]
  [-11.44490381   3.74009981]
  [ -9.71879295   3.74009981]
  [ -7.06943247   3.74009981]
  [ -4.5225345    3.74009981]
  [ -1.97563652   3.740

i am here
i am here
[[ 0.54067541  0.56708785  0.58647426  0.60070691  0.60879956  0.61414073
   0.61963615  0.62620323  0.63355106  0.64011493  0.6443887 ]]
i am here
[[[-8.46861657 -3.12104189]
  [-3.57286026 -3.12104189]
  [-3.45725293 -3.12104189]
  [-4.60213369 -3.12104189]
  [-5.48313305 -3.12104189]
  [-5.40706478 -3.12104189]
  [-4.35434688 -3.12104189]
  [-2.37902064 -3.12104189]
  [ 0.39492969 -3.12104189]
  [ 2.97355559 -3.12104189]
  [ 5.55218149 -3.12104189]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.54125591  0.56739836  0.58683784  0.60132064  0.60968267  0.61514842
   0.62057567  0.62686716  0.63373829  0.63981622  0.64359023]]
i am here
[[[-8.10397691 -3.03637137]
  [-3.12209978 -3.03637137]
  [-2.92787534 -3.03637137]
  [-4.00722026 -3.03637137]
  [-4.84046405 -3.03637137]
  [-4.73110027 -3.03637137]
  [-3.65555539 -3.03637137]
  [-1.66572961 -3.03637137]
  [ 1.11519682 -3.03637137]

...

i am here
[[[ 9.22443055 -6.52205164]
  [ 9.47339563 -6.52205164]
  [ 4.86981605 -6.52205164]
  [-0.38925109 -6.52205164]
  [-4.36934088 -6.52205164]
  [-6.50858304 -6.52205164]
  [-7.01260992 -6.52205164]
  [-6.06505886 -6.52205164]
  [-3.83440526 -6.52205164]
  [-1.42237086 -6.52205164]
  [ 0.98966354 -6.52205164]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.53743242  0.55016931  0.57651938  0.59666825  0.60469049  0.6066873
   0.60866053  0.61308416  0.62029531  0.62790843  0.63350418]]
i am here
[[[ 8.95709789 -6.99819744]
  [ 9.24075806 -6.99819744]
  [ 4.69489068 -6.99819744]
  [-0.50664637 -6.99819744]
  [-4.44021527 -6.99819744]
  [-6.54536608 -6.99819744]
  [-7.02537888 -6.99819744]
  [-6.06226487 -6.99819744]
  [-3.82385907 -6.99819744]
  [-1.4100448  -6.99819744]
  [ 1.00376947 -6.99819744]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstr

i am here
[[ 0.55167245  0.56509787  0.58841935  0.60576857  0.61244431  0.61407
   0.61596893  0.6202611   0.62717335  0.63447931  0.63970425]]
i am here
[[[ 5.33993076  1.00632307]
  [ 6.2817825   1.00632307]
  [ 2.63525939  1.00632307]
  [-1.70791414  1.00632307]
  [-4.96178612  1.00632307]
  [-6.57190488  1.00632307]
  [-6.70285049  1.00632307]
  [-5.51151111  1.00632307]
  [-3.15636551  1.00632307]
  [-0.71191634  1.00632307]
  [ 1.73253282  1.00632307]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.55231792  0.56589791  0.58884328  0.60584285  0.61236148  0.61398319
   0.61595529  0.62035609  0.62739122  0.63479914  0.64012354]]
i am here
[[[ 4.808294    0.483187  ]
  [ 5.84204117  0.483187  ]
  [ 2.28428539  0.483187  ]
  [-1.98290565  0.483187  ]
  [-5.18005856  0.483187  ]
  [-6.74961645  0.483187  ]
  [-6.85187127  0.483187  ]
  [-5.64128129  0.483187  ]
  [-3.27554459  0.483187  ]
  [-0.827407

i am here
[[ 0.54852511  0.56867249  0.58567876  0.59599082  0.59905341  0.60040725
   0.60408671  0.61147149  0.62238206  0.63328815  0.6421469 ]]
i am here
[[[ -7.2911955   -0.93653844]
  [ -4.71232594  -0.93653844]
  [ -6.77755491  -0.93653844]
  [ -9.76718237  -0.93653844]
  [-12.01066518  -0.93653844]
  [-12.8982359   -0.93653844]
  [-12.51808616  -0.93653844]
  [-10.98381802  -0.93653844]
  [ -8.44000352  -0.93653844]
  [ -5.9298578   -0.93653844]
  [ -3.41971209  -0.93653844]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.54765108  0.56842563  0.58522998  0.59524111  0.59815697  0.59953967
   0.6033645   0.61097704  0.62217789  0.63334603  0.64247107]]
i am here
[[[ -7.97299355   0.20350008]
  [ -5.30431503   0.20350008]
  [ -7.28279211   0.20350008]
  [-10.19816223   0.20350008]
  [-12.38621375   0.20350008]
  [-13.23414453   0.20350008]
  [-12.82595542   0.20350008]
  [-11.27287375   0.20350008]

[[ 0.52820056  0.55817143  0.57568585  0.58572041  0.58993724  0.59351704
   0.59968387  0.60941188  0.62231498  0.63457549  0.64475788]]
i am here
[[[-13.39757501  -2.43425442]
  [ -9.22502836  -2.43425442]
  [ -9.76961755  -2.43425442]
  [-11.46484322  -2.43425442]
  [-12.74687553  -2.43425442]
  [-12.95041025  -2.43425442]
  [-12.0893846   -2.43425442]
  [-10.23581991  -2.43425442]
  [ -7.52045218  -2.43425442]
  [ -4.95439944  -2.43425442]
  [ -2.3883467   -2.43425442]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.53191779  0.56099304  0.57902656  0.59005145  0.59510167  0.59902355
   0.60502141  0.6140789   0.62583445  0.63692684  0.6458902 ]]
i am here
[[[-12.35085868  -3.67249999]
  [ -8.09152854  -3.67249999]
  [ -8.55689567  -3.67249999]
  [-10.18608122  -3.67249999]
  [-11.4199905   -3.67249999]
  [-11.58997352  -3.67249999]
  [-10.70594523  -3.67249999]
  [ -8.83776944  -3.67249999]
  [ -6.11

i am here
[[ 0.52691055  0.56002791  0.58031186  0.5945648   0.60330187  0.60997419
   0.61714955  0.62556461  0.63480636  0.64287909  0.64867379]]
i am here
[[[-14.99456641  -2.49485871]
  [ -9.28586574  -2.49485871]
  [ -8.42814577  -2.49485871]
  [ -8.9543957   -2.49485871]
  [ -9.38460258  -2.49485871]
  [ -8.99423862  -2.49485871]
  [ -7.72604361  -2.49485871]
  [ -5.61384783  -2.49485871]
  [ -2.77404606  -2.49485871]
  [ -0.18128695  -2.49485871]
  [  2.41147215  -2.49485871]]] um
(1, 11, 2)
[ 3550.  4180.  4810.  5440.  6070.  6700.  7330.  7960.  8590.  9220.
  9850.] Angstrom
i am here
i am here
[[ 0.52462403  0.55862277  0.57907143  0.59334615  0.60215931  0.60899508
   0.61639228  0.62507136  0.63460564  0.6429233   0.64896762]]
i am here
[[[-15.75031178  -2.37979108]
  [ -9.95800398  -2.37979108]
  [ -9.02396156  -2.37979108]
  [ -9.48658851  -2.37979108]
  [ -9.87043371  -2.37979108]
  [ -9.44774606  -2.37979108]
  [ -8.15739035  -2.37979108]
  [ -6.03111824  -2.37979108]

TODO:
0. Fix the colorbar range to 0.5 - 0.7 for all plots
1. Get the same plots without blur+offset, can do without offset but need to create a new blur file with no blur in it
2. Get some histograms for the difference between center value and expected value for acceptance
3. Get 2D histograms for the distance of the maximum acceptance wrt to the center value acceptance
4. 

In [17]:
qsos_round = []
for i in range(num_source):
    qsos_round.append(generate_sources(1, 0., 0., seed=i))


In [142]:
print(*qsos_round)

(array([[ 0.,  0.]]), array([[ 0.45,  1.  ]]), array([[1, 1]]), array([[ 186.22723818,  340.90653737]])) (array([[ 0.,  0.]]), array([[ 0.45,  1.  ]]), array([[1, 1]]), array([[ 345.6062292 ,  251.82433798]]))
